# Übung SQL

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


In [ ]:
from IPython.display import Image

## 1. Setup für SQL

Die **IPython SQL magix extension** ermöglicht es, SQL-Abfragen direkt in Code-Zellen zu schreiben sowie die Ergebnisse direkt in **Pandas DataFrames** (was das heißt lernen wir noch) zu lesen. Dies funktioniert sowohl für die traditionellen Notebooks als auch für die modernen Jupyter Labs. Um sicherzugehen, dass alles funktioniert führt Ihr bitte folgenden Code aus:

In [ ]:
!pip install ipython-sql

Wenn Ihr mehr Informationen haben wollt, könnt Ihr gerne mal auf das GitHub Repository von **IPython SQL magic extension** schauen: https://github.com/catherinedevlin/ipython-sql

Als nächstes müsst Ihr das SQL-Modul laden:

In [ ]:
%load_ext sql

Der obige **Magic Command** lädt die ipython-sql-extension. Nun können wir eine Verbindung mit jeder Datenbank herstellen, die von **SQLAlchemy** unterstützt wird. In unserem Beispiel werden wir uns mit einer SQLite-Datenbank verbinden. Geben Sie den folgenden Befehl in die Codezelle ein:

In [ ]:
%sql sqlite://

**Aber warte, was ist ein Magic Command**?

**Magic Commands** sind eine Reihe von praktischen Funktionen in Jupyter Notebooks, die einige der häufigsten Probleme bei der Standard-Datenanalyse lösen sollen. Sie können alle verfügbaren Magics mit Hilfe von **%lsmagic** sehen.

In [ ]:
%lsmagic

**Aber warte, was ist SQLAlchemy?**

SQLAlchemy ist das Python-SQL-Toolkit und Object Relational Mapper, das Anwendungsentwicklern die volle Leistungsfähigkeit und Flexibilität von SQL bietet. Es ist eine vollständige Suite bekannter Persitence Pattern, die für einen effizienten und hochperformanten Datenbankzugriff entwickelt wurden, angepasst an eine einfache Python Domain Language: https://www.sqlalchemy.org/

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## 2. Erstellung einer eigenen Datenbank / Tabelle

Das erste was wir in unserer neuen Datenbank brauchen ist eine Tabelle - Tabellen erstellen wir mit einem **CREATE** Operand für welches wir auch das DB-Schema angeben müssen. Danach nutzen wir **INSERT**, damit wir zwei Einträge / Zustände hinzufügen können.

In [ ]:
%%sql sqlite://
CREATE TABLE IF NOT EXISTS EMPLOYEE (
    employee_id integer PRIMARY KEY,
    firstname varchar(50) NOT NULL,
    lastname varchar(50) NOT NULL);

Nun gucken wir uns an, ob diese Tabelle wirklich angelegt wurde.

In [ ]:
%%sql sqlite://
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

Lasst uns gemeinsam einen Blick auf die **[offizielle Dokumentation](https://www.sqlitetutorial.net/sqlite-create-table/)** werfen.

In [ ]:
%%sql
INSERT INTO EMPLOYEE VALUES(1, 'Pelle','John');  
INSERT INTO EMPLOYEE VALUES(2, 'Kai','Neubauer');

Hier auch noch einmal die offizielle Einführung für **[INSERT](https://www.sqlitetutorial.net/sqlite-insert/)**

Als nächstes sollten wir einmal prüfen, ob die Daten wirklich in den Tabellen vorhanden sind. Lasst uns den Output der Einträge ansehen....

In [ ]:
%%sql sqlite://
SELECT * FROM EMPLOYEE

Dokumentation für **[SELECT](https://www.sqlitetutorial.net/sqlite-select/)**

Mit dem folgenden Befehl löschen wir die Tabelle wieder

In [ ]:
%sql DROP TABLE EMPLOYEE

In [ ]:
%sql sqlite://  SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%';

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## 3. Laden und erste Analyse einer existierenden Datenbasis

Für die folgende Übung verwenden wir **SQL_SAFI-Datenbank (Studying African Farmer-led Irrigation (SAFI) database)**. Das SAFI-Projekt ist ein Forschungsprojekt, das sich mit den Bewässerungsmethoden der Bauern in Tansania und Mosambik beschäftigt. Dieser Datensatz besteht aus Umfragedaten, die sich auf Haushalte und Landwirtschaft beziehen.

In [ ]:
#Spezifizierung des Datepfades zum Laden der Datenbank
%sql sqlite:///src/SQL_SAFI.sqlite

Als ersten Schritt sollten wir uns einen Überblick der Tabellen verschaffen

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type ='table'; 

**AUFGABE**: Lasst und einen genaueren Blick auf die **Crops** Tabelle werfen und nur Zeilen auswählen, für welche **D_curr_crop** den Wert "maize" haben und die **ID kleiner, gleich 3** ist.

In [ ]:
%%sql 
SELECT * from Crops 
WHERE 
    D_curr_crop = "maize" AND ID <= 3;

**Ausblick:** Du kannst die selectierten Daten als Resultat speichern und anschließend in einen DataFrame für Deine weiteren Analysen und Visualisierungen übernehmen. 

In [ ]:
result = %sql SELECT * from Crops where ID <= 3;
crop_selection_df = result.DataFrame()
crop_selection_df.head()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## 4. Komplexere Joins über mehrere Tabellen

Wenn Ihr SQL zum ersten Mal anwendet, ist es üblich, dass Ihr mit Daten in einer einzigen Tabelle arbeiten. In der realen Welt haben Datenbanken in der Regel Daten in mehr als einer Tabelle. Wenn wir mit diesen Daten arbeiten wollen, müssen wir mehrere Tabellen in einer Abfrage kombinieren.

In [ ]:
#Spezifizierung des Datepfades zum Laden der Datenbank
%sql sqlite:///src/factbook.db

Wir werden eine Version der **Datenbank des CIA World Factbook (Factbook)** verwenden, die zwei Tabellen hat. Die erste Tabelle heißt **facts**, und jede Zeile steht für ein Land aus dem Factbook. Hier sind die ersten 5 Zeilen der **facts** Tabelle:

In [ ]:
%sql SELECT * FROM facts LIMIT 10;

Zusätzlich zur Faktentabelle gibt es eine zweite Tabelle mit dem Namen **cities**, die Informationen über die größten städtischen Gebiete der Länder im Factbook enthält. Werfen wir einen Blick auf die ersten Zeilen dieser Tabelle und eine Beschreibung dessen, was jede Spalte darstellt:

In [ ]:
%sql SELECT * FROM cities LIMIT 5;

Die letzte Spalte ist für uns von besonderem Interesse, da es sich um eine Spalte handelt, die auch in unserer ursprünglichen **facts** Tabelle existiert. Diese Verknüpfung zwischen den Tabellen ist wichtig, da sie verwendet wird, um die Daten in unseren Abfragen zu kombinieren. Unten sehen Sie ein Schemadiagramm, das die beiden Tabellen in unserer Datenbank, die Spalten darin und die Verknüpfung der beiden zeigt. Die Darstellung im Schemadiagramm zeigt deutlich die Verbindung zwischen der **Spalte id in der Tabelle facts** und der **Spalte facts_id in der Tabelle cities**.

![DB-Schema](./src/factsbook_db_schema.PNG)

Die häufigste Art, Daten mit SQL zu verknüpfen, ist ein **Inner Join**. Die Syntax für eine innere Verknüpfung lautet:
- **INNER JOIN**, der der SQL-Engine den Namen der Tabelle mitteilt, die Sie in Ihrer Abfrage verbinden möchten, und dass Sie eine innere Verknüpfung verwenden möchten
- **ON**, der der SQL-Engine mitteilt, welche Spalten für die Verknüpfung der beiden Tabellen verwendet werden sollen.

```sql
SELECT [column_names] FROM [table_name_one]
INNER JOIN [table_name_two] ON [join_constraint];
```


In [ ]:
%%sql
SELECT * FROM facts
/* Teilt SQL-Engine mit, dass Tabelle "cities" über Inner Join mit unserer Abfrage verbindet */
INNER JOIN cities
/* Das die Spalten cities.facts_id und facts.id das verbindende Element der Daten ist */
ON cities.facts_id = facts.id
LIMIT 5;

------------------
**Beispiel für United Arab Emirates**

In [ ]:
%%sql
SELECT * FROM facts
Where facts.id = 184;

Wir sehen, dass es lediglich eine Zeile für das Land United Arab Emirates gibt

In [ ]:
%%sql
SELECT * FROM cities
Where cities.facts_id = 184;

Wir sehen es gibt in der Tabelle cities 3 Städte: Abu Dhabi, Dubai und Sharjah

In [ ]:
%%sql
SELECT * FROM facts
INNER JOIN cities
ON cities.facts_id = facts.id
Where facts.id = 184
LIMIT 5;

Bei einem inner Join werden nun die drei Zeilen aus der cities genommen und die Informationen aus der Facts Tabelle für diese 3 Cities aufgeschrieben.

------------------

Eine innere Verknüpfung funktioniert, indem nur Zeilen aus jeder Tabelle einbezogen werden, die eine Übereinstimmung aufweisen, wie mit der ON-Klausel angegeben. Schauen wir uns in einem Diagramm an, wie unsere Verknüpfung aus dem vorherigen Bild funktioniert. Wir haben eine Auswahl von Zeilen eingefügt, die die Verknüpfung am besten veranschaulichen:

**Inner Join**
<div>
<img src="./src/inner-join.png" width="200"/>
</div>


![Inner_Join](./src/factsbook_db_innerjoin.PNG)

**Frage: Welche Zeilen werden durch den Inner Join nicht mit aufgenommen?**

Die Schreibweise kann auch noch etwas vereinfacht werden durch **[aliases](https://www.tutorialspoint.com/sqlite/sqlite_alias_syntax.htm)**

In [ ]:
%%sql
SELECT * FROM facts AS f
INNER JOIN cities AS c ON c.facts_id = f.id
LIMIT 5;

In [ ]:
%%sql
SELECT
/* SQL-Engine saves the aliases, so can select all columns from table c (cities) - declared a little later */
    c.*,
/* Select the name column of facts table and rename it to country name */
    f.name country_name
FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
LIMIT 5;

Wie bereits erwähnt, **schließt eine innere Verknüpfung keine Zeilen ein, für die es keine gegenseitige Übereinstimmung aus beiden Tabellen gibt**. Das bedeutet, dass es Informationen geben könnte, die wir in unserer Abfrage nicht sehen, wenn Zeilen nicht übereinstimmen. Lasst uns das einmal versuchen nachzuvollziehen:

In [ ]:
%sql SELECT COUNT(DISTINCT(name)) FROM facts;

In [ ]:
%sql SELECT COUNT(DISTINCT(facts_id)) FROM cities;

Wenn wir diese beiden Abfragen ausführen, können wir sehen, dass es einige Länder in der Faktentabelle gibt, die keine entsprechenden Städte in der Städtetabelle haben, was darauf hindeutet, dass wir möglicherweise unvollständige Daten haben.

Schauen wir uns an, wie wir eine Abfrage erstellen können, um die fehlenden Daten zu untersuchen, indem wir einen neuen Typ von Join verwenden - den linken Join.

Ein **linker Join umfasst alle Zeilen, die ein innerer Join auswählt, plus alle Zeilen aus der ersten (oder linken) Tabelle, die keine Übereinstimmung in der zweiten Tabelle haben**. Wir können dies als Venn-Diagramm darstellen.

**Left Join**
<div>
<img src="./src/left-join.png" width="200"/>
</div>


![Left_Join](./src/factsbook_db_leftjoin.PNG)'

In [ ]:
%%sql 
SELECT * FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id
LIMIT 50;

**Frage: Was würdet Ihr jetzt in der Tabelle erwarten?**

In [ ]:
%%sql 
SELECT
    f.id,
    f.name country,
    f.population,
    c.name
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id
WHERE c.name IS NULL;

In [ ]:
%%sql
/* 
Hat der Kosovo keine Größen Städte oder fehlen dort einfach Daten in der cities Tabelle?
*/

SELECT *
FROM cities
WHERE facts_id = 92

Wenn wir uns die Ergebnisse der Abfrage ansehen, können wir eine Reihe von **verschiedenen Gründen** erkennen, warum Länder keine entsprechenden Werte in Städten haben:

- Länder mit kleiner Bevölkerung und/oder ohne größere städtische Gebiete (die als Länder mit einer Bevölkerung von mehr als 750.000 Einwohnern definiert sind), z. B. San Marino, Kosovo und Nauru.
- Stadtstaaten, z. B. Monaco und Singapur.
- Territorien, die selbst keine Länder sind, z. B. Hongkong, Gibraltar und die Cookinseln.
- Regionen und Ozeane, die keine Länder sind, wie z. B. die Europäische Union und der Pazifische Ozean.
- Echte Fälle fehlender Daten, wie z. B. Taiwan.

### RIGHT JOIN und OUTER JOIN

Es gibt zwei weniger gebräuchliche Join-Typen, die von SQLite nicht unterstützt werden. Der erste ist ein **RIGHT JOIN**. Ein **RIGHT JOIN** ist, wie der Name schon sagt, genau das Gegenteil eines **LEFT JOIN** . Während der **LEFT JOIN**  alle Zeilen in der Tabelle vor der JOIN-Klausel einschließt, schließt der rechte Join alle Zeilen in der neuen Tabelle in der JOIN-Klausel ein.

In [ ]:
%%sql
SELECT f.name country, c.name city
FROM cities c
RIGHT JOIN facts f ON f.id = c.facts_id
LIMIT 5;

Der andere Join-Typ, der von SQLite nicht unterstützt wird, ist ein **FULL OUTER JOIN**. Ein Full Outer Join schließt alle Zeilen aus den Tabellen auf beiden Seiten des Joins ein.

In [ ]:
%%sql
SELECT f.name country, c.name city
FROM cities c
FULL OUTER JOIN facts f ON f.id = c.facts_id
LIMIT 5;

![Venn_Join](./src/factsbook_db_venn_joins.PNG)'

Zuletzt kannst Du natürlich die Ergebnisse einer SQL-Abfrage in einer **neuen Tabelle speichern**, wenn Du viele weitere Abfragen darauf aufbauen möchtest. **Frage**: Wann kann das sinnvoll sein Redundante Daten zu haben?

In [ ]:
%%sql
CREATE TABLE special_country 
AS SELECT
    f.name country,
    f.population
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id
WHERE c.name IS NULL;

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type ='table'; 

In [ ]:
%sql SELECT * FROM special_country LIMIT 10;

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## 5. Aufgaben zum Üben

In [ ]:
import IPython.display

**1. Erstelle eine Abfrage mit Ländern und ihren Hauptstädten aus unserer Datenbank. Tipp: Unser erster Schritt ist, darüber nachzudenken, welche Spalten wir in unserer endgültigen Abfrage benötigen.**

**2. Erstelle eine Abfrage, um die Hauptstädte und deren korrespondierenden Länder mit der größten Population zu identifizieren (ersten 10). Tipp: Für dieses Vorgehen brauchst Du zum einen JOIN, eine WHERE-Klausel und nach der Identifikation der richtigen Spalten den Operand [ORDER BY](https://www.sqlitetutorial.net/sqlite-order-by/)**

**3. Erstelle eine geordnete Liste von Ländern die die höchste Einwohner_pro_qm rate haben und zeige das Land, die Einwohneranzahl, die Fläche des Landes und die Einwohner_pro_qm**

**4. Berechne den Anteil der Einwohneranzahl der größten Städte pro land an der Gesamtbevölkerung**

**(OPTIONAL) 5. Erstelle eine Abfrage mit Nicht-Hauptstädte mit mehr als 10 Millionen Einwohnern inklusive der Länderinformationen. Diese Aufgabe erfordert das Problem, ähnlich einem Algorithmus erst einmal in mehrere Schritte zun unterteilen und mit [SUBQUERIES](https://www.tutorialspoint.com/sql/sql-sub-queries.htm) zu arbeiten.**

**Wenn Ihr Lust habt noch weiter einzusteigen, kann ich Euch z. B. diese Websiten / Artikel empfehlen:**
- [Tutorial SQPoint](https://www.tutorialspoint.com/sqlite/)
- [A Complete Guide to SQL for Data Science](https://pub.towardsai.net/a-complete-guide-to-sql-for-data-science-35743e73fd)

Dies sind aber nur einige wenige von vielen Möglichkeiten, die Euch dabei helfen können zum SQL-Experten zu werden.